## Code to search the contents for all of your SO answers

- File with
  - Tags
  - Title
  - ViewCount
  - Score
  - Answer ID
  - Accepted Answer Id
  - Answer Body

In [ ]:
import pandas as pd
from IPython.display import HTML

In [ ]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

- Perform query at [StackExchange Data Explorer][sede] with the following query, using your Stack Overflow user ID

```sql
SELECT 
    Ans.CreationDate, Qus.Tags, Qus.Title, Qus.ViewCount,
    Ans.Score, Ans.ID, Qus.AcceptedAnswerId
FROM Posts as Qus
LEFT JOIN Posts as Ans
ON Qus.ID = Ans.ParentID
WHERE Ans.OwnerUserId = ##UserId##
ORDER BY Ans.CreationDate DESC
```

- Save the file as `'so_QueryResults.csv'`

  [sede]: https://data.stackexchange.com/stackoverflow/query/new

In [ ]:
# load file
so = pd.read_csv('so_QueryResults.csv', dtype={'ID': int, 'AcceptedAnswerId': str})

In [ ]:
so.CreationDate = pd.to_datetime(so.CreationDate).dt.date

In [ ]:
so.AcceptedAnswerId.fillna(value=0, inplace=True)  # replace NaN with 0
so.AcceptedAnswerId = so.AcceptedAnswerId.astype(int)  # convert from float to int
so['Accepted'] = so.ID == so.AcceptedAnswerId  # add a Boolean column True if your answer is the accepted answer

In [ ]:
# using the answer id, create a link to your answer
so['Answer Link'] = so[['ID', 'Title']].apply(lambda x: f'<a href="https://stackoverflow.com/questions/{x[0]}">{x[1]}</a>', axis=1)

In [ ]:
so.drop(columns=['ID', 'AcceptedAnswerId', 'Title'], inplace=True)

In [ ]:
so.Tags = so.Tags.str.replace('><', ' ').str.replace('<|>', '').str.split()

In [ ]:
so.head()

In [ ]:
# sort by ViewCount
# so = so.sort_values(['ViewCount'], ascending=False)  # ['Accepted', 'Score', 'ViewCount']  
# so.head(100)

In [ ]:
# sort by Accepted and CreationDate
# so = so.sort_values(['Accepted', 'CreationDate'], ascending=False).reset_index(drop=True)
# so.head(100)

In [ ]:
# various filters for search the Body of the answer
son = so[so.Body.str.lower().str.contains('matplotlib', regex=False)]
son = son[son.Body.str.lower().str.contains('bar', regex=False)]
son = son[son.Body.str.lower().str.contains('patch(color', regex=False)]
son

In [ ]:
so = so.drop(columns=['Body'])  # drop Body of the html will be a mess

In [ ]:
HTML(so.to_html('so_answers.html', escape=False))  # html to a file

In [ ]:
HTML(so.to_html(escape=False))  # html displayed in notebook